In [1]:
import pandas as pd
import numpy as np

In [2]:
urls = dict(
    pers_auto = "https://www.casact.org/sites/default/files/2021-04/ppauto_pos.csv"
    , wc = "https://www.casact.org/sites/default/files/2021-04/wkcomp_pos.csv"
    , comm_auto = "https://www.casact.org/sites/default/files/2021-04/comauto_pos.csv"
    , med_mal = "https://www.casact.org/sites/default/files/2021-04/medmal_pos.csv"
    , products = "https://www.casact.org/sites/default/files/2021-04/prodliab_pos.csv"
    , other_liab = "https://www.casact.org/sites/default/files/2021-04/othliab_pos.csv"
)

In [3]:
pers_auto = "https://www.casact.org/sites/default/files/2021-04/ppauto_pos.csv"

In [4]:
rawdf = pd.read_csv(urls['wc'])
rawdf.head()

,GRCODE,GRNAME,AccidentYear,DevelopmentYear,DevelopmentLag,IncurLoss_D,CumPaidLoss_D,BulkLoss_D,EarnedPremDIR_D,EarnedPremCeded_D,EarnedPremNet_D,Single,PostedReserve97_D
0,86,Allstate Ins Co Grp,1988,1988,1,367404,70571,127737,400699,5957,394742,0,281872
1,86,Allstate Ins Co Grp,1988,1989,2,362988,155905,60173,400699,5957,394742,0,281872
2,86,Allstate Ins Co Grp,1988,1990,3,347288,220744,27763,400699,5957,394742,0,281872
3,86,Allstate Ins Co Grp,1988,1991,4,330648,251595,15280,400699,5957,394742,0,281872
4,86,Allstate Ins Co Grp,1988,1992,5,354690,274156,27689,400699,5957,394742,0,281872


In [11]:
%run "C:\Users\AndyW\OneDrive\Documents\GitHub\cic\naicschp\data.py"
df, gpdf = read_naic_csv(lob='wc')
df

,group_code,type_of_loss,ay,ep,1,2,3,4,5,6,7,8,9,10
0,86,bulk_reserve,1988,394742,127737.0,60173.0,27763.0,15280.0,27689.0,20641.0,14513.0,15862.0,8974.0,8843.0
1,86,bulk_reserve,1989,374252,116172.0,52298.0,20543.0,28846.0,22033.0,17174.0,17851.0,10305.0,12520.0,NaN
2,86,bulk_reserve,1990,280320,108959.0,62543.0,32561.0,25858.0,21189.0,16645.0,9428.0,11861.0,NaN,NaN
3,86,bulk_reserve,1991,313982,112836.0,60251.0,37066.0,28931.0,24175.0,12850.0,12899.0,NaN,NaN,NaN
4,86,bulk_reserve,1992,252698,73135.0,52160.0,39554.0,20231.0,12532.0,11114.0,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6595,44300,reported_loss,1993,0,0.0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN
6596,44300,reported_loss,1994,0,0.0,0.0,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN
6597,44300,reported_loss,1995,387,280.0,169.0,310.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6598,44300,reported_loss,1996,1168,481.0,693.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [14]:
lobs = ['pers_auto', 'wc', 'comm_auto', 'med_mal', 'products', 'other_liab']
dflist = []
for lob in lobs:
    df, _ = read_naic_csv(lob=lob)
    df['lob'] = lob
    dflist.append(df)
df = pd.concat(dflist)
df

KeyError: "['EarnedPremDIR_H1', 'EarnedPremCeded_H1', 'PostedReserve97_H1'] not found in axis"

In [12]:
rpt_paid_loss = df.loc[df.type_of_loss.isin(['paid_loss', 'reported_loss']) & df.]

0      bulk_reserve
10     case_reserve
20    incurred_loss
30        paid_loss
40    reported_loss
Name: type_of_loss, dtype: object

adding standardization to the triangles:
* normalize by group code and accident year
* this preserves the development year pattern, and should make it clear to a model that cumulative goes from low to high

In [35]:
## standardize each row of the triangle by the mean and std of the group and accident year

means = df.drop(columns='cy dev_lag'.split()).groupby('group_code ay'.split()).mean()
stds = df.drop(columns='cy dev_lag'.split()).groupby('group_code ay'.split()).std()

df1 = df.merge(means, on='group_code ay'.split(), suffixes=('', '_mean')).merge(stds, on='group_code ay'.split(), suffixes=('', '_std'))

for c in 'incurred_loss paid_loss bulk_reserve case_reserve reported_loss ep'.split():
    df1[f"{c}_norm"] = (df1[c] - df1[f'{c}_mean']) / df1[f'{c}_std']

df1['group_code ay dev_lag'.split() + [f'{c}_norm' for c in 'incurred_loss paid_loss bulk_reserve case_reserve reported_loss'.split()]].pivot_table(index='group_code ay'.split(), columns='dev_lag', values='reported_loss_norm')

dev_lag                1         2         3         4         5         6   \
group_code ay                                                                 
86         1988 -2.633626 -0.503698  0.059916 -0.080296  0.312076  0.394712   
           1989 -2.352242 -0.273629 -0.571907  0.205090  0.361887  0.382181   
           1990 -2.362159  0.048414 -0.071730  0.155099  0.278090  0.490617   
           1991 -1.920985 -0.652887 -0.041968  0.388066  0.520551  0.815417   
           1992 -1.927641 -0.123495  0.146894  0.475047  0.700432  0.728763   
...                   ...       ...       ...       ...       ...       ...   
44091      1994 -1.147546 -0.525148  0.771514  0.901180       NaN       NaN   
           1995 -1.104748  0.261429  0.843319       NaN       NaN       NaN   
           1996 -0.707107  0.707107       NaN       NaN       NaN       NaN   
44300      1995  0.363507 -1.130909  0.767403       NaN       NaN       NaN   
           1996 -0.707107  0.707107       NaN       NaN       NaN       NaN   

dev_lag                7         8         9        10  
group_code ay                                           
86         1988  0.490638  0.523254  0.722964  0.71406  
           1989  0.550321  0.827301  0.870999      NaN  
           1990  0.696327  0.765342       NaN      NaN  
           1991  0.891806       NaN       NaN      NaN  
           1992       NaN       NaN       NaN      NaN  
...                   ...       ...       ...      ...  
44091      1994       NaN       NaN       NaN      NaN  
           1995       NaN       NaN       NaN      NaN  
           1996       NaN       NaN       NaN      NaN  
44300      1995       NaN       NaN       NaN      NaN  
           1996       NaN       NaN       NaN      NaN  

[807 rows x 10 columns]

In [20]:
df.pivot_table(index='group_code ay ep'.split(), columns='dev_lag', values='paid_loss')

dev_lag                      1         2         3         4         5   \
group_code ay   ep                                                        
86         1988 394742  70571.0  155905.0  220744.0  251595.0  274156.0   
           1989 374252  66547.0  136447.0  179142.0  211343.0  231430.0   
           1990 280320  52233.0  133370.0  178444.0  204442.0  222193.0   
           1991 313982  59315.0  128051.0  169793.0  196685.0  213165.0   
           1992 252698  39991.0   89873.0  114117.0  133003.0  154362.0   
...                         ...       ...       ...       ...       ...   
44300      1993 0           0.0       0.0       0.0       0.0       0.0   
           1994 0           0.0       0.0       0.0       0.0       NaN   
           1995 387        16.0     119.0     156.0       NaN       NaN   
           1996 1168       70.0     184.0       NaN       NaN       NaN   
           1997 1013      123.0       NaN       NaN       NaN       NaN   

dev_lag                       6         7         8         9         10  
group_code ay   ep                                                        
86         1988 394742  287676.0  298499.0  304873.0  321808.0  325322.0  
           1989 374252  244750.0  254557.0  270059.0  273873.0       NaN  
           1990 280320  232940.0  253337.0  256788.0       NaN       NaN  
           1991 313982  234676.0  239195.0       NaN       NaN       NaN  
           1992 252698  159496.0       NaN       NaN       NaN       NaN  
...                          ...       ...       ...       ...       ...  
44300      1993 0            NaN       NaN       NaN       NaN       NaN  
           1994 0            NaN       NaN       NaN       NaN       NaN  
           1995 387          NaN       NaN       NaN       NaN       NaN  
           1996 1168         NaN       NaN       NaN       NaN       NaN  
           1997 1013         NaN       NaN       NaN       NaN       NaN  

[1320 rows x 10 columns]

In [33]:
# get incurred loss triangles
incurred_loss = (
    df
    .drop(columns='paid_loss bulk_reserve case_reserve report_loss'.split())
    # .query(f'cy <= {df.ay.max()}')
    .query('cy <= ay.max()')
    .pivot_table(index=['group_code', 'ay', 'ep'], columns='dev_lag', values='incurred_loss')
)

incurred_loss.loc[43]

,dev_lag,1,2,3,4,5,6,7,8,9,10
ay,ep,,,,,,,,,,
1988,895,607.0,647.0,582.0,598.0,614.0,615.0,615.0,614.0,614.0,614.0
1989,3407,2254.0,2859.0,2979.0,2764.0,2826.0,3017.0,2990.0,2978.0,2978.0,NaN
1990,5889,5843.0,7424.0,8307.0,8671.0,9066.0,8926.0,8767.0,8765.0,NaN,NaN
1991,16784,11422.0,16093.0,18648.0,19095.0,18653.0,18469.0,18407.0,NaN,NaN,NaN
1992,27647,19933.0,24162.0,28739.0,28329.0,28071.0,27722.0,NaN,NaN,NaN,NaN
1993,35138,24604.0,32130.0,33575.0,32769.0,33722.0,NaN,NaN,NaN,NaN,NaN
1994,42605,40735.0,43944.0,42511.0,41712.0,NaN,NaN,NaN,NaN,NaN,NaN
1995,47629,43064.0,43705.0,42909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,46929,41837.0,41304.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# convert cumulative triangle to incremental triangle, leaving the first column as is
inc_incurred_loss = incurred_loss.copy()
inc_incurred_loss.iloc[:, 1:] = incurred_loss.iloc[:, 1:].values - incurred_loss.iloc[:, :-1].values
inc_incurred_loss.loc[43]

,dev_lag,1,2,3,4,5,6,7,8,9,10
ay,ep,,,,,,,,,,
1988,895,607.0,40.0,-65.0,16.0,16.0,1.0,0.0,-1.0,0.0,0.0
1989,3407,2254.0,605.0,120.0,-215.0,62.0,191.0,-27.0,-12.0,0.0,NaN
1990,5889,5843.0,1581.0,883.0,364.0,395.0,-140.0,-159.0,-2.0,NaN,NaN
1991,16784,11422.0,4671.0,2555.0,447.0,-442.0,-184.0,-62.0,NaN,NaN,NaN
1992,27647,19933.0,4229.0,4577.0,-410.0,-258.0,-349.0,NaN,NaN,NaN,NaN
1993,35138,24604.0,7526.0,1445.0,-806.0,953.0,NaN,NaN,NaN,NaN,NaN
1994,42605,40735.0,3209.0,-1433.0,-799.0,NaN,NaN,NaN,NaN,NaN,NaN
1995,47629,43064.0,641.0,-796.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,46929,41837.0,-533.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [32]:
inc_incurred_loss

dev_lag                     1       2       3      4      5      6      7   \
group_code ay   ep                                                           
43         1988 895      607.0    40.0   -65.0   16.0   16.0    1.0    0.0   
           1989 3407    2254.0   605.0   120.0 -215.0   62.0  191.0  -27.0   
           1990 5889    5843.0  1581.0   883.0  364.0  395.0 -140.0 -159.0   
           1991 16784  11422.0  4671.0  2555.0  447.0 -442.0 -184.0  -62.0   
           1992 27647  19933.0  4229.0  4577.0 -410.0 -258.0 -349.0    NaN   
...                        ...     ...     ...    ...    ...    ...    ...   
43494      1993 6687    5873.0   169.0    66.0 -163.0  -10.0    NaN    NaN   
           1994 6557    5103.0   572.0    86.0  -28.0    NaN    NaN    NaN   
           1995 8582    6508.0  -380.0   192.0    NaN    NaN    NaN    NaN   
           1996 10184   6554.0   353.0     NaN    NaN    NaN    NaN    NaN   
           1997 9625    5542.0     NaN     NaN    NaN    NaN    NaN    NaN   

dev_lag                  8    9    10  
group_code ay   ep                     
43         1988 895    -1.0  0.0  0.0  
           1989 3407  -12.0  0.0  NaN  
           1990 5889   -2.0  NaN  NaN  
           1991 16784   NaN  NaN  NaN  
           1992 27647   NaN  NaN  NaN  
...                     ...  ...  ...  
43494      1993 6687    NaN  NaN  NaN  
           1994 6557    NaN  NaN  NaN  
           1995 8582    NaN  NaN  NaN  
           1996 10184   NaN  NaN  NaN  
           1997 9625    NaN  NaN  NaN  

[1460 rows x 10 columns]

In [30]:
inc_incurred_loss.loc[43].cumsum(axis=1)

,dev_lag,1,2,3,4,5,6,7,8,9,10
ay,ep,,,,,,,,,,
1988,895,607.0,647.0,582.0,598.0,614.0,615.0,615.0,614.0,614.0,614.0
1989,3407,2254.0,2859.0,2979.0,2764.0,2826.0,3017.0,2990.0,2978.0,2978.0,NaN
1990,5889,5843.0,7424.0,8307.0,8671.0,9066.0,8926.0,8767.0,8765.0,NaN,NaN
1991,16784,11422.0,16093.0,18648.0,19095.0,18653.0,18469.0,18407.0,NaN,NaN,NaN
1992,27647,19933.0,24162.0,28739.0,28329.0,28071.0,27722.0,NaN,NaN,NaN,NaN
1993,35138,24604.0,32130.0,33575.0,32769.0,33722.0,NaN,NaN,NaN,NaN,NaN
1994,42605,40735.0,43944.0,42511.0,41712.0,NaN,NaN,NaN,NaN,NaN,NaN
1995,47629,43064.0,43705.0,42909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,46929,41837.0,41304.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [31]:
incurred_loss.loc[43]

,dev_lag,1,2,3,4,5,6,7,8,9,10
ay,ep,,,,,,,,,,
1988,895,607.0,647.0,582.0,598.0,614.0,615.0,615.0,614.0,614.0,614.0
1989,3407,2254.0,2859.0,2979.0,2764.0,2826.0,3017.0,2990.0,2978.0,2978.0,NaN
1990,5889,5843.0,7424.0,8307.0,8671.0,9066.0,8926.0,8767.0,8765.0,NaN,NaN
1991,16784,11422.0,16093.0,18648.0,19095.0,18653.0,18469.0,18407.0,NaN,NaN,NaN
1992,27647,19933.0,24162.0,28739.0,28329.0,28071.0,27722.0,NaN,NaN,NaN,NaN
1993,35138,24604.0,32130.0,33575.0,32769.0,33722.0,NaN,NaN,NaN,NaN,NaN
1994,42605,40735.0,43944.0,42511.0,41712.0,NaN,NaN,NaN,NaN,NaN,NaN
1995,47629,43064.0,43705.0,42909.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1996,46929,41837.0,41304.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
